In [86]:
import pandas as pd
import numpy as np

In [87]:
df = pd.read_pickle(
    "../../data/paired_spectra/casmi2022/processed_massformer/spec_df.pkl"
)
df_h = df[df["prec_type"] == "[M+H]+"]
print(len(df_h))
df_h.head()
smiles_h = df_h["smiles"].to_list()

170


In [88]:
df_h["name"] = "casmi2022_" + df_h["casmi_id"].astype(str)
df_h.reset_index(drop=True, inplace=True)
df_h.head()

/var/folders/n5/t_ds6vws3ns1lyk9130hfx180000gn/T/ipykernel_93451/3196511267.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_h["name"] = "casmi2022_" + df_h["casmi_id"].astype(str)


,casmi_id,ion_mode,priority,smiles,prec_type,rt,prec_mz,spec_type,peaks,ace,nce,inst_type,frag_mode,res,spec_id,group_id,spec_id_old,mol_id_old,mol_id,name
0,3,P,True,COc1ccc(-c2oc3c(CC=C(C)C)c(O)cc(O)c3c(=O)c2OC2...,[M+H]+,5.5959,719.253867,MS2,"[(50.65100861, 3705.4970703125), (52.45254517,...",NaN,NaN,FT,HCD,4,0,2,2,2,1196787,casmi2022_3
1,9,P,True,CN(C)CCc1c[nH]c2ccc(CS(=O)(=O)N3CCCC3)cc12,[M+H]+,3.4133,336.173416,MS2,"[(50.28293991, 49982.94921875), (50.28532791, ...",NaN,NaN,FT,HCD,4,3,8,8,8,1196790,casmi2022_9
2,10,P,True,Cn1c(CNc2ccc(C(=N)N)cc2)nc2cc(C(=O)N(CCC(=O)O)...,[M+H]+,2.2642,472.208380,MS2,"[(53.03887939, 6174.09765625), (53.56915283, 5...",NaN,NaN,FT,HCD,4,4,9,9,9,1196791,casmi2022_10
3,11,P,True,CCCC(=O)OCC(OC(=O)CCC)C(OC(=O)CCC)C(Cn1c2nc(=O...,[M+H]+,6.0879,657.311672,MS2,"[(55.05422974, 4195.5727539063), (55.98180008,...",NaN,NaN,FT,HCD,4,5,10,10,10,1196792,casmi2022_11
4,15,P,True,CC1CCC2(C(=O)O)CCC3(C)C(=CCC4C5(C)CC(O)C(O)C(C...,[M+H]+,4.8524,519.331159,MS2,"[(58.7617836, 2441.0568847656), (67.05400848, ...",NaN,NaN,FT,HCD,4,6,14,14,14,1196793,casmi2022_15


In [89]:
for smi in smiles_h:
    print(smi)

COc1ccc(-c2oc3c(CC=C(C)C)c(O)cc(O)c3c(=O)c2OC2OC(C)C(OC(C)=O)C(OC3OC(CO)C(O)C(O)C3O)C2O)cc1
CN(C)CCc1c[nH]c2ccc(CS(=O)(=O)N3CCCC3)cc12
Cn1c(CNc2ccc(C(=N)N)cc2)nc2cc(C(=O)N(CCC(=O)O)c3ccccn3)ccc21
CCCC(=O)OCC(OC(=O)CCC)C(OC(=O)CCC)C(Cn1c2nc(=O)[nH]c(=O)c-2nc2cc(C)c(C)cc21)OC(=O)CCC
CC1CCC2(C(=O)O)CCC3(C)C(=CCC4C5(C)CC(O)C(O)C(C)(C(=O)O)C5CCC43C)C2C1(C)O
CCC1(C)C(=O)C=Cc2c1oc(C)c(C)c2=O
Cc1ccc(C2(c3ccc(C)c(O)c3)OC(=O)c3ccccc32)cc1O
Cc1cc(SCc2sc(-c3ccc(C(F)(F)F)cc3)nc2C)ccc1OCC(=O)O
CCCc1nn(C)c2c(=O)[nH]c(-c3cc(S(=O)(=O)N4CCN(CC)CC4)ccc3OCC)nc12
CC(C)NC(=O)CCc1ccccc1
CCCc1nn(C)c2c(=O)[nH]c(-c3cc(S(=O)(=O)N4CCN(CCO)CC4)ccc3OCC)nc12
CCCCCCCCCCCCN(CCCCCCCCCCCC)CCCCCCCCCCCC
CC1=C(C(=O)O)CC2(C)CCC3C(C)(O)CCC132
CCCc1oc(=O)ccc1C(O)CO
COC(=O)c1cc(OC)c2c(c1-c1c(C(=O)OC)cc(OC)c3c1OCO3)OCO2
COc1cc(O)c2c(=O)oc(C)c(C)c2c1C
CC(=CC(=O)O)CCC1(C)C(C)CCC2(C)C(C)=CCCC21
COc1cc2nccc3c4ccccc4n(c1=O)c23
CCC(O)C(O)CC=CCC=CCCCC(N)=O
C=c1[nH]cc(CO)c(=CNNC(=O)c2ccncc2)c1=O
COc1cc2c(cc1OC)CN(CCc1ccc(NC(=O)c3cccc4c

In [90]:
cls_file = "../../data/paired_spectra/casmi2022/classyfire_casmi.csv"
df_cls = pd.read_csv(cls_file)
df_cls.head(40)

,CompoundID,ChemOntID,ClassifiedResults
0,Q11600375-1,CHEMONTID:0000000,Kingdom: Organic compounds
1,Q11600375-1,CHEMONTID:0000261,Superclass: Phenylpropanoids and polyketides
2,Q11600375-1,CHEMONTID:0000334,Class: Flavonoids
3,Q11600375-1,CHEMONTID:0001111,Subclass: Flavonoid glycosides
4,Q11600375-1,CHEMONTID:0001583,Intermediate_nodes: Flavonoid O-glycosides
5,Q11600375-1,CHEMONTID:0003531,Direct_parent: Flavonoid-3-O-glycosides
6,Q11600375-1,CHEMONTID:0003504,alternative_parents: 8-prenylated flavones
7,Q11600375-1,CHEMONTID:0002596,alternative_parents: 4'-O-methylated flavonoids
8,Q11600375-1,CHEMONTID:0002992,alternative_parents: 5-hydroxyflavonoids
9,Q11600375-1,CHEMONTID:0002993,alternative_parents: 7-hydroxyflavonoids


In [91]:
# remove rows where ChemOntID is NotAvailable
df_cls = df_cls[df_cls["ChemOntID"] != "NotAvailable"]
df_cls.head()

,CompoundID,ChemOntID,ClassifiedResults
0,Q11600375-1,CHEMONTID:0000000,Kingdom: Organic compounds
1,Q11600375-1,CHEMONTID:0000261,Superclass: Phenylpropanoids and polyketides
2,Q11600375-1,CHEMONTID:0000334,Class: Flavonoids
3,Q11600375-1,CHEMONTID:0001111,Subclass: Flavonoid glycosides
4,Q11600375-1,CHEMONTID:0001583,Intermediate_nodes: Flavonoid O-glycosides


In [92]:
# split classifiedResults column on :
df_cls[["level", "value"]] = df_cls["ClassifiedResults"].str.split(
    ": ", expand=True
)
# remove rows where level is alternative parent
df_cls = df_cls[df_cls["level"] != "alternative_parents"]
df_cls.head(20)

,CompoundID,ChemOntID,ClassifiedResults,level,value
0,Q11600375-1,CHEMONTID:0000000,Kingdom: Organic compounds,Kingdom,Organic compounds
1,Q11600375-1,CHEMONTID:0000261,Superclass: Phenylpropanoids and polyketides,Superclass,Phenylpropanoids and polyketides
2,Q11600375-1,CHEMONTID:0000334,Class: Flavonoids,Class,Flavonoids
3,Q11600375-1,CHEMONTID:0001111,Subclass: Flavonoid glycosides,Subclass,Flavonoid glycosides
4,Q11600375-1,CHEMONTID:0001583,Intermediate_nodes: Flavonoid O-glycosides,Intermediate_nodes,Flavonoid O-glycosides
5,Q11600375-1,CHEMONTID:0003531,Direct_parent: Flavonoid-3-O-glycosides,Direct_parent,Flavonoid-3-O-glycosides
39,Q11600375-2,CHEMONTID:0000000,Kingdom: Organic compounds,Kingdom,Organic compounds
40,Q11600375-2,CHEMONTID:0000002,Superclass: Organoheterocyclic compounds,Superclass,Organoheterocyclic compounds
41,Q11600375-2,CHEMONTID:0000211,Class: Indoles and derivatives,Class,Indoles and derivatives
42,Q11600375-2,CHEMONTID:0000183,Subclass: Tryptamines and derivatives,Subclass,Tryptamines and derivatives


In [93]:
# compress df_cls on CompoundID by creating column for each level
levels = df_cls["level"].unique()
ids = [int(id.split("-")[-1]) - 1 for id in df_cls["CompoundID"].unique()]
df_comp = pd.DataFrame(index=ids, columns=levels)

for i, row in df_cls.iterrows():
    df_comp.loc[int(row["CompoundID"].split("-")[-1]) - 1, row["level"]] = row[
        "value"
    ]

df_comp.head(20)

,Kingdom,Superclass,Class,Subclass,Intermediate_nodes,Direct_parent
0,Organic compounds,Phenylpropanoids and polyketides,Flavonoids,Flavonoid glycosides,Flavonoid O-glycosides,Flavonoid-3-O-glycosides
1,Organic compounds,Organoheterocyclic compounds,Indoles and derivatives,Tryptamines and derivatives,NaN,Tryptamines and derivatives
2,Organic compounds,Organoheterocyclic compounds,Benzimidazoles,NaN,NaN,Benzimidazoles
3,Organic compounds,Organoheterocyclic compounds,Pteridines and derivatives,Alloxazines and isoalloxazines,NaN,Flavins
4,Organic compounds,Lipids and lipid-like molecules,Prenol lipids,Triterpenoids,NaN,Triterpenoids
5,Organic compounds,Organoheterocyclic compounds,Pyrans,Pyranones and derivatives,NaN,Pyranones and derivatives
6,Organic compounds,Organoheterocyclic compounds,Benzofurans,Benzofuranones,NaN,Benzofuranones
7,Organic compounds,Benzenoids,Benzene and substituted derivatives,Phenoxyacetic acid derivatives,NaN,Phenoxyacetic acid derivatives
8,Organic compounds,Benzenoids,Benzene and substituted derivatives,Benzenesulfonamides,NaN,Benzenesulfonamides
9,Organic compounds,Benzenoids,Benzene and substituted derivatives,NaN,NaN,Benzene and substituted derivatives


In [94]:
# insert rows to make id increment by 1
missing_ids = set(np.arange(1, len(df_h), 1)) - set(df_comp.index.astype(int))
missing_df = pd.DataFrame(index=list(missing_ids), columns=levels)
df_complete = pd.concat([df_comp, missing_df])
df_complete.index = df_complete.index.astype(int)
df_complete.sort_index(inplace=True)

print(len(df_complete))
df_complete.head(30)

170


,Kingdom,Superclass,Class,Subclass,Intermediate_nodes,Direct_parent
0,Organic compounds,Phenylpropanoids and polyketides,Flavonoids,Flavonoid glycosides,Flavonoid O-glycosides,Flavonoid-3-O-glycosides
1,Organic compounds,Organoheterocyclic compounds,Indoles and derivatives,Tryptamines and derivatives,NaN,Tryptamines and derivatives
2,Organic compounds,Organoheterocyclic compounds,Benzimidazoles,NaN,NaN,Benzimidazoles
3,Organic compounds,Organoheterocyclic compounds,Pteridines and derivatives,Alloxazines and isoalloxazines,NaN,Flavins
4,Organic compounds,Lipids and lipid-like molecules,Prenol lipids,Triterpenoids,NaN,Triterpenoids
5,Organic compounds,Organoheterocyclic compounds,Pyrans,Pyranones and derivatives,NaN,Pyranones and derivatives
6,Organic compounds,Organoheterocyclic compounds,Benzofurans,Benzofuranones,NaN,Benzofuranones
7,Organic compounds,Benzenoids,Benzene and substituted derivatives,Phenoxyacetic acid derivatives,NaN,Phenoxyacetic acid derivatives
8,Organic compounds,Benzenoids,Benzene and substituted derivatives,Benzenesulfonamides,NaN,Benzenesulfonamides
9,Organic compounds,Benzenoids,Benzene and substituted derivatives,NaN,NaN,Benzene and substituted derivatives


In [101]:
# join df_h and df_complete on index
casmi_cls = df_h.join(df_complete, how="left")
print(len(casmi_cls))
casmi_cls.head()

170


,casmi_id,ion_mode,priority,smiles,prec_type,rt,prec_mz,spec_type,peaks,ace,...,spec_id_old,mol_id_old,mol_id,name,Kingdom,Superclass,Class,Subclass,Intermediate_nodes,Direct_parent
0,3,P,True,COc1ccc(-c2oc3c(CC=C(C)C)c(O)cc(O)c3c(=O)c2OC2...,[M+H]+,5.5959,719.253867,MS2,"[(50.65100861, 3705.4970703125), (52.45254517,...",NaN,...,2,2,1196787,casmi2022_3,Organic compounds,Phenylpropanoids and polyketides,Flavonoids,Flavonoid glycosides,Flavonoid O-glycosides,Flavonoid-3-O-glycosides
1,9,P,True,CN(C)CCc1c[nH]c2ccc(CS(=O)(=O)N3CCCC3)cc12,[M+H]+,3.4133,336.173416,MS2,"[(50.28293991, 49982.94921875), (50.28532791, ...",NaN,...,8,8,1196790,casmi2022_9,Organic compounds,Organoheterocyclic compounds,Indoles and derivatives,Tryptamines and derivatives,NaN,Tryptamines and derivatives
2,10,P,True,Cn1c(CNc2ccc(C(=N)N)cc2)nc2cc(C(=O)N(CCC(=O)O)...,[M+H]+,2.2642,472.208380,MS2,"[(53.03887939, 6174.09765625), (53.56915283, 5...",NaN,...,9,9,1196791,casmi2022_10,Organic compounds,Organoheterocyclic compounds,Benzimidazoles,NaN,NaN,Benzimidazoles
3,11,P,True,CCCC(=O)OCC(OC(=O)CCC)C(OC(=O)CCC)C(Cn1c2nc(=O...,[M+H]+,6.0879,657.311672,MS2,"[(55.05422974, 4195.5727539063), (55.98180008,...",NaN,...,10,10,1196792,casmi2022_11,Organic compounds,Organoheterocyclic compounds,Pteridines and derivatives,Alloxazines and isoalloxazines,NaN,Flavins
4,15,P,True,CC1CCC2(C(=O)O)CCC3(C)C(=CCC4C5(C)CC(O)C(O)C(C...,[M+H]+,4.8524,519.331159,MS2,"[(58.7617836, 2441.0568847656), (67.05400848, ...",NaN,...,14,14,1196793,casmi2022_15,Organic compounds,Lipids and lipid-like molecules,Prenol lipids,Triterpenoids,NaN,Triterpenoids


In [107]:
casmi_cls[levels] = casmi_cls[levels].fillna("Unclassified")

In [108]:
casmi_cls.to_pickle(
    "../../data/paired_spectra/casmi2022/casmi_classyfire_results.pkl"
)

In [109]:
casmi_cls["Superclass"].value_counts()

Superclass
Lipids and lipid-like molecules              39
Organoheterocyclic compounds                 32
Phenylpropanoids and polyketides             31
Unclassified                                 22
Benzenoids                                   20
Organic acids and derivatives                10
Organic oxygen compounds                      6
Organic nitrogen compounds                    5
Alkaloids and derivatives                     4
Lignans, neolignans and related compounds     1
Name: count, dtype: int64

In [110]:
casmi_cls["Class"].value_counts()

Class
Prenol lipids                             25
Unclassified                              22
Benzene and substituted derivatives       16
Flavonoids                                14
Indoles and derivatives                    9
Carboxylic acids and derivatives           8
Fatty Acyls                                8
Organooxygen compounds                     6
Steroids and steroid derivatives           6
Organonitrogen compounds                   5
Tannins                                    4
Pyrans                                     3
Isocoumarins and derivatives               3
Stilbenes                                  3
Benzopyrans                                3
Isoflavonoids                              2
Phenanthrenes and derivatives              2
Azoles                                     2
Naphthalenes                               2
Quinolines and derivatives                 2
Cinnamic acids and derivatives             2
Benzofurans                                2
Benz